In [9]:
import torch
import torchaudio as ta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ffmpeg
import IPython
import librosa
import sklearn
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
ta.set_audio_backend("sox_io")

In [10]:
number = 5
data_paths = './dataset/'
data_path = data_paths + "voice/s{}.m4a".format(number)
data_path_out = data_paths + "voice/s{}.wav".format(number)
labels_path = data_paths + "key_and_time/s{}.txt".format(number)
detections_path = data_paths + "detections/s{}.txt".format(number)

In [11]:
waveform, sample_rate = ta.load(data_path_out)
labels = pd.read_csv(labels_path, delimiter="\"", header=0, usecols=[3, 7], \
                          names=['key', 'time']).dropna()
detections = pd.read_csv(detections_path, index_col=0)
labels['time'] /= 1000

In [75]:
detections['start_sample'] = detections['start'] * sample_rate
detections['stop_sample'] = detections['stop'] * sample_rate
labels['time_sample'] = labels['time']/1000 * sample_rate

In [252]:
da_features = []
da_labels = []
seq_lens = []
for number in range(12):
    d_set_size = 0
    data_paths = './dataset/'
    data_path = data_paths + "voice/s{}.m4a".format(number)
    data_path_out = data_paths + "voice/s{}.wav".format(number)
    labels_path = data_paths + "key_and_time/s{}.txt".format(number)
    labels = pd.read_csv(labels_path, delimiter="\"", header=0, usecols=[3, 7], \
                          names=['key', 'time']).dropna()
    labels['time_sample'] = labels['time']/1000 * sample_rate
    detections_path = data_paths + "detections/s{}.txt".format(number)
    detections = pd.read_csv(detections_path, index_col=0)
    detections['start_sample'] = detections['start'] * sample_rate
    detections['stop_sample'] = detections['stop'] * sample_rate
    for i in range(len(detections)):
        start = int(detections['start_sample'][i])
        stop = int(detections['stop_sample'][i])
        labels['dist_to_start'] = np.abs(labels['time_sample'] - start)
        min_idx = labels['dist_to_start'].idxmin()
        
        label = labels['key'][min_idx]
        
        keypress = np.array(waveform[:, start:stop])
        length = stop - start
        n_mels = 10
        features = librosa.feature.mfcc(y=keypress, sr=sample_rate, n_mfcc=n_mels, hop_length=1, n_fft=int(length/5))
        #print(features.shape)
        #if features.shape == (1, n_mels, 2):
        da_features.append(features.flatten()[:n_mels*2395])
        da_labels.append(label)
        number_det += 1
        d_set_size += 1
#         else: 
#             print('fail')
#             
    seq_lens.append(d_set_size)


/Users/sierrawyllie/PythonEnvs/324Project/lib/python3.8/site-packages/librosa/feature/spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/Users/sierrawyllie/PythonEnvs/324Project/lib/python3.8/site-packages/librosa/feature/spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/Users/sierrawyllie/PythonEnvs/324Project/lib/python3.8/site-packages/librosa/feature/spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
/Users/sierra

In [253]:
da_features = np.array(da_features).reshape(-1, n_mels*2395)

In [254]:
da_features.shape

(3756, 23950)

In [255]:
da_labels = pd.DataFrame(da_labels, columns=['true label'])

In [256]:
n_samples = da_features.shape[0]
X_train = da_features[:int(.8*n_samples), :]
y_train = da_labels[:int(.8*n_samples)]
X_val = da_features[int(.8*n_samples):, :]
y_val = da_labels[int(.8*n_samples):]

In [257]:
from sklearn.ensemble import RandomForestClassifier

In [258]:
model = RandomForestClassifier(n_estimators=300)

In [259]:
X_train.shape

(3004, 23950)

In [260]:
labs_forest = y_train['true label'].to_numpy().flatten()
vlabs_forest = y_val['true label'].to_numpy().flatten()

In [261]:
model.fit(X_train, labs_forest)

RandomForestClassifier(n_estimators=300)

In [263]:
model.score(X_val, y_val)

0.12898936170212766